在深度学习中，我们通常会频繁地对数据进行操作。作为动手学深度学习的基础，本节将介绍如何对内存中的数据进行操作。

在TensorFlow中，tensor是一个类，也是存储和变换数据的主要工具。如果你之前用过NumPy，你会发现tensor和NumPy的多维数组非常类似。然而，**tensor提供GPU计算和自动求梯度等更多功能**，这些使tensor更加适合深度学习。

###  2.2.1 创建 tensor

我们先介绍`tensor`的最基本功能，我们用arange函数创建一个行向量。

In [1]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
print(tf.__version__)

2.4.1


In [2]:
x = tf.constant(list(range(12)))

print(x.shape)

(12,)


In [3]:
x

<tf.Tensor: shape=(12,), dtype=int32, numpy=array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])>

In [4]:
x.shape

TensorShape([12])

In [5]:
X = tf.reshape(x,(3,4))
X

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])>

注意X属性中的形状发生了变化。上面`x.reshape((3, 4))`也可写成`x.reshape((-1, 4))`或`x.reshape((3, -1))`。由于`x`的元素个数是已知的，这里的-1是能够通过元素个数和其他维度的大小推断出来的。

接下来，我们创建一个各元素为0，形状为(2, 3, 4)的张量。实际上，之前创建的向量和矩阵都是特殊的张量。

In [6]:
tf.zeros((2,3,4))

<tf.Tensor: shape=(2, 3, 4), dtype=float32, numpy=
array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]], dtype=float32)>

In [7]:
Y = tf.constant([[2,1,4,3],[1,2,3,4],[4,3,2,1]])
Y

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[2, 1, 4, 3],
       [1, 2, 3, 4],
       [4, 3, 2, 1]])>

有些情况下，我们需要随机生成`tensor`中每个元素的值。下面我们创建一个形状为(3, 4)的`tensor`。它的每个元素都随机采样于均值为0、标准差为1的正态分布。

In [8]:
R = tf.random.normal(shape=[3,4],mean=0, stddev=1.0 )
R

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[-0.8494363 , -0.8918495 , -0.24627087,  0.43391666],
       [-0.2354259 , -0.8252366 , -0.59108466,  0.01672698],
       [ 0.46052375, -0.61473143,  2.0484073 , -1.1296929 ]],
      dtype=float32)>

###  在 TF1版本中，需要 RUN 才能看到里面具体的数值

## 2.2.2 运算

`tensor`支持大量的运算符（operator）。例如，我们可以对之前创建的两个形状为(3, 4)的`tensor`做按元素加法。所得结果形状不变。

In [10]:
X + Y

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[ 2,  2,  6,  6],
       [ 5,  7,  9, 11],
       [12, 12, 12, 12]])>

In [11]:
Y = tf.cast(Y, tf.float32)

对`tensor`中的所有元素求和得到只有一个元素的`tensor`。

## 2.2.3 广播机制

前面我们看到如何对两个形状相同的`tensor`做按元素运算。当对两个形状不同的`tensor`按元素运算时，可能会触发广播（broadcasting）机制：先适当复制元素使这两个`tensor`形状相同后再按元素运算。

定义两个`tensor`：

In [12]:
A = tf.reshape(tf.constant(list(range(3))), (3,1))
B = tf.reshape(tf.constant(list(range(2))), (1,2))
A, B

(<tf.Tensor: shape=(3, 1), dtype=int32, numpy=
 array([[0],
        [1],
        [2]])>,
 <tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[0, 1]])>)

In [14]:
A + B

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[0, 1],
       [1, 2],
       [2, 3]])>

## 2.2.4 索引

在`tensor`中，索引（index）代表了元素的位置。`tensor`的索引从0开始逐一递增。例如，一个3行2列的矩阵的行索引分别为0、1和2，列索引分别为0和1。

在下面的例子中，我们指定了`tensor`的行索引截取范围[1:3]。依据左闭右开指定范围的惯例，它截取了矩阵`X`中行索引为1和2的两行。

In [15]:
ans = X[1:3]
ans

<tf.Tensor: shape=(2, 4), dtype=int32, numpy=
array([[ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])>

我们可以指定`tensor`中需要访问的单个元素的位置，如矩阵中行和列的索引，并为该元素重新赋值。

In [16]:
X = tf.Variable(X)
X[1,2].assign(9)


<tf.Variable 'UnreadVariable' shape=(3, 4) dtype=int32, numpy=
array([[ 0,  1,  2,  3],
       [ 4,  5,  9,  7],
       [ 8,  9, 10, 11]])>

In [20]:
X

<tf.Variable 'Variable:0' shape=(3, 4) dtype=int32, numpy=
array([[ 0,  1,  2,  3],
       [ 4,  5,  9,  7],
       [ 8,  9, 10, 11]])>

## 2.2.5 运算的内存开销

在前面的例子里我们对每个操作新开内存来存储运算结果。举个例子，即使像`Y = X + Y`这样的运算，我们也会新开内存，然后将`Y`指向新内存。为了演示这一点，我们可以使用Python自带的`id`函数：如果两个实例的ID一致，那么它们所对应的内存地址相同；反之则不同。

In [21]:
X = tf.Variable(X)
Y = tf.cast(Y, dtype=tf.int32)

before = id(Y)
Y = Y + X
id(Y) == before

False

In [22]:
X

<tf.Variable 'Variable:0' shape=(3, 4) dtype=int32, numpy=
array([[ 0,  1,  2,  3],
       [ 4,  5,  9,  7],
       [ 8,  9, 10, 11]])>